In [ ]:
#!pip install kaggle-environments -U

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
print(device)

In [ ]:
import argparse
import glob
import os
import random
from typing import Callable

from stable_baselines3 import DQN  # pip install stable-baselines3
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.vec_env import SubprocVecEnv

from importlib import reload
import agent_policy
reload(agent_policy) # Reload the file from disk incase the above agent-writing cell block was edited
from agent_policy import AgentPolicy

from luxai2021.env.agent import Agent
from luxai2021.env.lux_env import LuxEnvironment
from luxai2021.game.constants import LuxMatchConfigs_Default


# Default Lux configs
configs = LuxMatchConfigs_Default

# Create a default opponent agent
opponent = AgentPolicy(mode="inference", model=model)

# Create a RL agent in training mode
player = AgentPolicy(mode="train")

# Create the Lux environment
env = LuxEnvironment(configs=configs,
                     learning_agent=player,
                     opponent_agent=opponent)

# Define the model, you can pick other RL algos from Stable Baselines3 instead if you like
model = DQN("MlpPolicy",
                env,
                verbose=1,
                tensorboard_log="./lux_tensorboard2/",
                learning_rate=0.001,
                gamma=0.999,
                #gae_lambda=0.95,
                batch_size=2048 * 8,
                #n_steps=2048 * 8
            )

# Define a learning rate schedule
# (number of steps, learning_rate)
schedule = [
    #(2000000, 0.01),
    (10000000, 0.001),
    (10000000, 0.0001),
    #(600, 0.001),
    #(600, 0.0001),
]

In [ ]:
from stable_baselines3.common.utils import get_schedule_fn

print("Training model...")
run_id = 1

# Save a checkpoint every 1M steps
checkpoint_callback = CheckpointCallback(save_freq=1000000, #1000000
                                         save_path='./models_2/',
                                         name_prefix=f'rl_model_{run_id}')

# Train the policy
for steps, learning_rate in schedule:
    model.lr_schedule = get_schedule_fn(learning_rate)
    model.learn(total_timesteps=steps,
                callback=checkpoint_callback,
                reset_num_timesteps = False)

# Save final model
model.save(path=f'models_2/model_2.zip')

print("Done training model.")

In [ ]:
# Move the dependent packages into kaggle submissions
!mv luxai2021 kaggle_submissions
!mv stable_baselines3 kaggle_submissions
!rm ./kaggle_submissions/agent_policy.py
!cp agent_policy.py kaggle_submissions

# Copy the agent and model to the submission 
!cp ./agent_policy.py kaggle_submissions
!cp ./models_2/model_2.zip kaggle_submissions

!ls kaggle_submissions

In [ ]:
!tar -czf submission_2.tar.gz -C kaggle_submissions .
!ls